<a href="https://colab.research.google.com/github/PaulToronto/Learning_Manim/blob/main/02_Tutorials_Manims_Building_Blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manim's Building Blocks

https://docs.manim.community/en/stable/tutorials/building_blocks.html

In [1]:
# 1. Install System Dependencies (Includes LaTeX for MathTex)
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev ffmpeg

# 2. Install Manim and specific IPython version for stability
!pip install manim IPython==8.21.0

# 3. Restart kernel (Run this once, then comment it out)
# exit()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
63 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree

## Imports

In [5]:
from manim import *
import numpy as np

## `Mobject`: `manim.mobject.mobject.Mobject`

- `Mobject` is the base class of all objects that can be displayed
- Bases: `object`
- Every `Mobject` has a `points` attribute, wich is a NumPy array. Even an "empty" one has a location in $\mathbb{R}^{3}$
- The `points` Numpy array has 3 columns, (x, y, z) and a row for each point
- `get_center()`
    - In Manim, the **center** is the geometric center of the **Bounding Box** that encloses all the points in the mobject

$$C = \left( \frac{x_{max} + x_{min}}{2}, \frac{y_{max} + y_{min}}{2}, \frac{z_{max} + z_{min}}{2} \right)$$

- You can apply `.shift()`, `rotate()` and `scale()` to it. It will update it's internal coordinate matrix even if there is nothing visual to move.

In [26]:
%%manim -ql -v WARNING TestMobject

class TestMobject(Scene):
    def construct(self):
        self.add(NumberPlane())

        # Create invisible Mobject and a visible dot to track it
        # Note that it needs `points` to be able to calculate the center
        my_Mobject = Mobject().set(points=np.array([ORIGIN]))
        dot = Dot(color=RED)

        # Link the dot to the Mobject's position
        dot.add_updater(lambda d: d.move_to(my_Mobject.get_center()))

        self.add(dot)
        # Print the center BEFORE the shift
        print(f"Before: {my_Mobject.get_center()}")
        self.wait(3)

        # move my_Mobject
        my_Mobject.shift(RIGHT * 3 + UP * 2)
        # a hack to get the .add_updater() to work
        dot.update()
        # Print the center AFTER the shift
        print(f"After: {my_Mobject.get_center()}")
        self.wait(3)


Manim Community v0.19.2

Before: [0. 0. 0.]
After: [3. 2. 0.]


In [40]:
%%manim -ql -v WARNING TestMobject2

class TestMobject2(Scene):
    def construct(self):
        self.add(NumberPlane())

        # the Math Object is the source of truth
        my_Mobject = Mobject().set(points=np.array([ORIGIN]))

        # the Dot is the visual follower
        dot = Dot(color=RED)
        dot.add_updater(lambda d: d.move_to(my_Mobject.get_center()))
        self.add(dot)

        # the Text Box for real time display
        # always redraw ensures this function run every frame
        coord_label = always_redraw(lambda:
                Text(f"Center: {my_Mobject.get_center()}")
                .to_edge(UP)
                .set_color(YELLOW)
        )

        self.add(coord_label)

        self.wait(3)

        # move the Math Object
        my_Mobject.shift(RIGHT * 3 + UP * 2)
        dot.update() # poke the dot
        coord_label.update() # poke the label

        self.wait(3)


Manim Community v0.19.2